In [ ]:
import cv2
import numpy as np
from sklearn.svm import SVC
import os
import sys
import time


# EXTRACTOR 1

In [ ]:
def binarioPro(ul,u,ur,l,pix,r,dl,d,dr):
    res = np.zeros(pix.shape)
    vecinos = np.array([ul,u,ur,r,dr,d,dl,l])
    for i in range(len(vecinos)):
        res = res + np.power(2,i)*(vecinos[i] >=pix)
    return res

In [ ]:
def feature_extraction(imagen, N, R):
    imagen_amp = cv2.copyMakeBorder(imagen, 1, 1, 1, 1, cv2.BORDER_REPLICATE)
    newIm = np.zeros(imagen.shape)
    #for i in range(1,len(imagen)+1):
     #   for j in range(1,len(imagen[0])+1):
      #      newIm[i-1,j-1] = binarioPro(imagen_amp[i-1,j-1], imagen_amp[i-1,j], imagen_amp[i-1,j+1], imagen_amp[i,j-1],imagen_amp[i,j], imagen_amp[i,j+1],
       #     imagen_amp[i+1,j-1], imagen_amp[i+1,j], imagen_amp[i+1,j+1])

    newIm = binarioPro(imagen_amp[0:-2,0:-2],imagen_amp[0:-2,1:-1],imagen_amp[0:-2,2:],imagen_amp[1:-1,0:-2],imagen_amp[1:-1,1:-1],imagen_amp[1:-1,2:],imagen_amp[2:,0:-2],imagen_amp[2:,1:-1],imagen_amp[2:,2:])
    
    height, width = imagen.shape[:2]

  # Calculamos el número de bloques en filas y columnas
    num_blocks_rows = height // N
    num_blocks_cols = width // N

    # Dividimos la imagen en bloques de 16 filas
    blocks = np.vsplit(imagen, num_blocks_rows)

    # Creamos una lista para guardar todas las celdas   
    histogramas = np.zeros((num_blocks_rows*num_blocks_cols,256))
    
    # Iteramos sobre los bloques
    for i, block in enumerate(blocks):
        # Dividimos cada bloque en bloques d
        # e 16 columnas
        cells = np.hsplit(block, num_blocks_cols)

        # Iteramos sobre las celdas
        for j, cell in enumerate(cells):
            # Añadimos cada celda a la lista    
            histogramas[j*i] = cv2.calcHist([cell], [0], None, [256], [0,256]).flatten()

        
    return histogramas.flatten()

In [ ]:
#MALARDO

datasets = ["cat_dog_100" ,"cat_dog_500"  ]

N = 16
R = 3

HEIGHT =  384
WIDTH = 528

num_blocks_rows = HEIGHT // N
num_blocks_cols = WIDTH // N

for dataset in datasets:
    print("dataset: ", dataset)
    # Iniciamos el contador de tiempo
    startTimer = time.perf_counter()

    # Definimos el directorio de entrenamiento y prueba
    train_dir = dataset+"/"+"train"
    test_dir = dataset+"/"+"test"

    # Inicializamos los arrays de datos de entrenamiento y prueba
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    print("Procesamos las imágenes de entrenamiento")
    # Procesamos las imágenes de entrenamiento
    for label in os.listdir(train_dir):
        label_dir = os.path.join(train_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path , 0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)            
            
            
            features = feature_extraction(resized_image, N, R);       

            X_train.append(features.flatten())
            y_train.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtrain ... (" + str(int(progress)) + " %)")
        print("")

    print("Procesamos las imágenes de prueba")
    # Procesamos las imágenes de prueba
    for label in os.listdir(test_dir):
        label_dir = os.path.join(test_dir, label)
        print("Procesando label: ", label)
        for count, image_filename in enumerate(os.listdir(label_dir)):
            image_path = os.path.join(label_dir, image_filename)
            image = cv2.imread(image_path,0)
            resized_image = cv2.resize(image, (HEIGHT, WIDTH), interpolation=cv2.INTER_LINEAR)
                
            features = feature_extraction(resized_image, N, R);   

            
            X_test.append(features.flatten())
            y_test.append(label)

            progress = 100 * (count + 1) / len(os.listdir(label_dir))
            sys.stdout.write(
                "\r Procesando Xtest ... (" + str(int(progress)) + " %)")
        print("")
    print("")
    
    # Convertimos los datos de entrenamiento y prueba a arrays de NumPy
    X_train = np.array(X_train, np.uint8)
    y_train = np.array(y_train)
    X_test = np.array(X_test, np.uint8)
    y_test = np.array(y_test)


    print("SHAPE Xtrin",X_train.shape)
    print("SHAPE y_train",y_train.shape)
    print("SHAPE X_test",X_test.shape)
    print("SHAPE y_test",y_test.shape)


    # Detenemos el contador de tiempo
    endTimer = time.perf_counter()

    # Calculamos el tiempo transcurrido
    elapsed = endTimer - startTimer
    # Obtenemos el tiempo transcurrido en minutos y segundos
    minutes, seconds = divmod(elapsed, 60)

    # Mostramos el tiempo transcurrido en minutos y segundos
    print(f"Tiempo transcurrido: {minutes:.0f} minutos {seconds:.2f} segundos")
    
    # Creamos un clasificador SVM
    clf = SVC(gamma='auto')

    # Entrenamos el clasificador con los datos de entrenamiento
    clf.fit(X_train, y_train)

    # Realizamos predicciones con el clasificador entrenado
    y_pred = clf.predict(X_test)

    # Calculamos el porcentaje de acierto
    accuracy = np.mean(y_pred == y_test)
    print("Precisión: {:.2f}%".format(accuracy * 100))


    
    
